### Imports

In [123]:
import pandas as pd
import numpy as np

import folium
import os
from tqdm import tqdm

In [104]:
DATA_FOLDER = './'
raw_loc = pd.read_csv(os.path.join(DATA_FOLDER, '서울시 정류장마스터 정보.csv'))
raw_data = pd.read_csv(os.path.join(DATA_FOLDER, '정류장별 총 승차 승객수 정보.csv'))


In [150]:
raw_loc

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부
0,500000288,행사장입구,마을버스,25965.0,126.691556,37.321742,미설치
1,500000287,행사장입구,마을버스,25095.0,126.691728,37.321942,미설치
2,500000286,아남중전기,일반차로,25963.0,126.710485,37.319934,미설치
3,500000285,3Z,일반차로,25964.0,126.707130,37.320566,미설치
4,500000284,굿스틸,일반차로,25961.0,126.702906,37.323147,미설치
...,...,...,...,...,...,...,...
66233,100000005,혜화동로터리.여운형활동터,중앙차로,1005.0,127.001744,37.586243,설치
66234,100000004,종로2가.삼일교,중앙차로,1004.0,126.987613,37.568579,설치
66235,100000003,명륜3가.성대입구,중앙차로,1003.0,126.998340,37.582671,설치
66236,100000002,창경궁.서울대학교병원,중앙차로,1002.0,126.996566,37.579183,설치


In [149]:
raw_loc = raw_loc.drop(raw_loc[raw_loc['정류장_유형'] == '가상정류장'].index).reset_index(drop = True)
raw_data = raw_data.drop(raw_data[raw_data['노선_ID'] == 0].index).reset_index(drop = True)
raw_data = raw_data.drop(raw_data[raw_data['노선_ID'].isna()].index).reset_index(drop = True)

data=raw_data.sort_values(by=['노선_ID','기준_날짜','정류장_순서',])
data

,기준_날짜,노선_ID,정류장_ID,승차_승객_수,승차_승객_수_00시,승차_승객_수_01시,승차_승객_수_02시,승차_승객_수_03시,승차_승객_수_04시,승차_승객_수_05시,...,승차_승객_수_15시,승차_승객_수_16시,승차_승객_수_17시,승차_승객_수_18시,승차_승객_수_19시,승차_승객_수_20시,승차_승객_수_21시,승차_승객_수_22시,승차_승객_수_23시,정류장_순서
624626,20220305,100100001,101000331.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
624646,20220305,100100001,101000053.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,8.0,6.0,15.0,2.0,6.0,NaN,NaN,NaN,2.0
624643,20220305,100100001,101000054.0,118.0,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,9.0,4.0,5.0,3.0,4.0,NaN,NaN,NaN,3.0
624642,20220305,100100001,101000103.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,3.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,4.0
624627,20220305,100100001,101000293.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,4.0,2.0,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,20220312,241461015,232000433.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
9,20220312,241461015,232000432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
4,20220312,241461015,232000843.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0
7,20220312,241461015,232000569.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0


In [154]:
data['승차_승객_수'].max()

4026.0

In [ ]:
bus_id = raw_loc['정류장_ID']
bus_x = raw_loc['경도']
bus_y = raw_loc['위도']


#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
long = raw_loc['경도'].mean()
lat = raw_loc['위도'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 9)

coords = []
for i in range(len(raw_loc)-1):
    x = bus_x[i]
    y = bus_y[i]
    coords.append([x, y])
    
for i in range(len(coords)):
    folium.Circle(
        location = coords[i],
        radius = 50,
        color = '#000000',
        fill = 'crimson',
    ).add_to(m)

m.save('C:\Code\Python\Data_Science\Bus\\bus.html')

In [119]:
bus_id = raw_loc['정류장_ID']
bus_x = raw_loc['경도']
bus_y = raw_loc['위도']


#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
long = raw_loc['경도'].mean()
lat = raw_loc['위도'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 9)

coords = []
for i in range(len(raw_loc)-1):
    x = bus_x[i]
    y = bus_y[i]
    coords.append([x, y])
    
for i in range(len(coords)):
    folium.Circle(
        location = coords[i],
        radius = 50,
        color = '#000000',
        fill = 'crimson',
    ).add_to(m)

m.save('C:\Code\Python\Data_Science\Bus\\bus.html')